In [0]:
# Importando módulo de seguimiento
import sys
import importlib
sys.path.append('/Workspace/Users/luisfdiaz@bcp.com.pe/ScoreGuardian')

import core.SegScore as rmm
importlib.reload(rmm)

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType

#Mostrar los duplicados
def print_res(sparkf_df):
  import pandas as pd
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  pd_spark_df = sparkf_df.toPandas()
 
  return pd_spark_df

In [0]:
# Importando base y configuraciones iniciales
base_seguimiento = spark.sql("select distinct * from catalog_lhcl_prod_bcp.bcp_edv_fabseg.t39290_hm_app_hippyme")
base_seguimiento = base_seguimiento.withColumn("CODMES", col("CODMES").cast("integer"))
base_seguimiento = base_seguimiento.withColumn("DEF5", when(col("DISTANCIA_DEF24") <= 5, lit(1)).otherwise(lit(0)))
base_seguimiento = base_seguimiento.withColumn("DEF4", when(col("DISTANCIA_DEF24") <= 4, lit(1)).otherwise(lit(0))) 

base_seguimiento.persist(StorageLevel.MEMORY_AND_DISK).count()

# Verificando duplicandos en el segundo Join
dup = base_seguimiento.groupBy("CODCLAVEOPECTA", "CODMES").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
fc = pd.DataFrame([(1, 1), (1, 1), (1, 1)], columns=['CODMES', 'FAC_CAL_DRI_N'])
fm = pd.DataFrame([(1, 1), (1, 1), (1, 1)], columns=['CODMES', 'FAC_CAL_DRI_M'])

In [0]:
base_seguimiento = base_seguimiento.withColumn("ACT_HIP", when(col("CODMES")<202308, "ANTES").otherwise(col("ACT_HIP")))
base_seguimiento = base_seguimiento.dropDuplicates(subset=['CODMES', 'CODCLAVEOPECTA'])

In [0]:
pd_rbm='PD_RBM'
pd_ant='PD_BASE_ESPEJO' 
pd_cal='PD_APP_HIP_TRDPYME_2Q24'
pd_trc='XB_ADC_POND85_F'
tipo_banda = 'Jeffrey'#Jeffrey Vasiseck
codmes_default = 202307
columna_monto='MTOAPROBADO_SOLES'
titulo='Calibración Applicant App Tradicional'
condiciones= "CODMES >= 201901 AND DEF_120_BLOQ_REF_0 = 0"

In [0]:
# Objeto de Seguimiento
Obj_Seg = rmm.MonitorScore(
  nombre_modelo         = 'Personas Modelo Applicant Pyme',
  codigo_modelo         = 'MOD-BCP-20658',
  tier                  = 'II',
  detalle_seguimiento   = 'Seguimiento Applicant Pyme',
  mes_seguimiento       = '202504',
  base_monitoreo        = base_seguimiento,
  pd1                   = pd_cal,
  pd2                   = pd_trc,
  monto_credito         = columna_monto,
  query_univ            = "DEF_120_BLOQ_REF_0=0 and codmes >202201 AND PD_BASE_ESPEJO IS NOT NULL and ACT_HIP IN ('EMISION MEDIO DE PAGO', 'DESEMBOLSO', 'ANTES')",
  bandas                = tipo_banda,
  codmes_default        = codmes_default,
  meses_ventana_target  = 24,
  meses_atraso_target   = 4
)

# Módulo Hipotecario

## LTV Hipotecaria (*)

Esta variable tuvo correcciones en DWH y LHCL. Se comentó con el negocio sobre su importancia

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('LTV_CREADA', col('MTOAPROBADO_SOLES')/col('CSOL_ValorComVivienda'))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('LTV_SALESFORCE', when(col('CODMES')<=202502, col('LTV_SALESFORCE')).otherwise(col('LTV_CREADA')))

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("LTV_CORREGIDO_VF", when(col("CODMES") >= 202308,  col("NFT_LTV")).otherwise(col("LTV_CORREGIDO_0")))

# Cortamos la variable
plazo = Obj_Seg.CutVar(variables=['LTV_CORREGIDO_VF', 'LTV_SUITE'], num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

In [0]:
driver = 'LTV_CORREGIDO_VF_Q'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08,
  metrica_secundaria = 'Media'
)

In [0]:
driver = 'LTV_SUITE_Q'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
  'LTV_SUITE_Q',
  when((col('LTV_SUITE') >= 0) & (col('LTV_SUITE') <= 0.40), "0. [0, 40%]")
  .when((col('LTV_SUITE') > 0.4) & (col('LTV_SUITE') <= 0.60), "1. (40%, 60%]")
  .when((col('LTV_SUITE') > 0.6) & (col('LTV_SUITE') <= 0.70), "2. (60%, 70%]")
  .when((col('LTV_SUITE') > 0.7) & (col('LTV_SUITE') <= 0.80), "3. (70%, 80%]")
  .when((col('LTV_SUITE') > 0.8) & (col('LTV_SUITE') <= 0.90), "4. (80%, 90%]")
  .when((col('LTV_SUITE') > 0.9) & (col('LTV_SUITE') <= 1), "5. (90%, 100%]")
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
  'LTV_SALESFORCE_Q',
  when((col('LTV_CORREGIDO_VF') >= 0) & (col('LTV_CORREGIDO_VF') <= 0.40), "0. [0, 40%]")
  .when((col('LTV_CORREGIDO_VF') > 0.4) & (col('LTV_CORREGIDO_VF') <= 0.60), "1. (40%, 60%]")
  .when((col('LTV_CORREGIDO_VF') > 0.6) & (col('LTV_CORREGIDO_VF') <= 0.70), "2. (60%, 70%]")
  .when((col('LTV_CORREGIDO_VF') > 0.7) & (col('LTV_CORREGIDO_VF') <= 0.80), "3. (70%, 80%]")
  .when((col('LTV_CORREGIDO_VF') > 0.8) & (col('LTV_CORREGIDO_VF') <= 0.90), "4. (80%, 90%]")
  .when((col('LTV_CORREGIDO_VF') > 0.9) & (col('LTV_CORREGIDO_VF') <= 1), "5. (90%, 100%]")
)

psi_dri = Obj_Seg.MixPSIVar(
  # Parametros de figura individual
    query_uso         ='CODMES >= 202011',
    cast_int          =True,
    codmes_inicio1    =202011,
    codmes_fin1       =202204,
    codmes_inicio2    =202501,
    codmes_fin2       =202504,
    dim_grafico       =(25, 6),
    pos_leyenda       =(0.5, -0.35),
 
    # Parametros de figura final
    tipo_vista        = 'M',
    ncolumnas         = 2,
    nfilas            = 1,
    variables         = ['LTV_SUITE_Q', 'LTV_SALESFORCE_Q'],
    dim_grafico_total = (17, 5),
    ncol_leyenda      =5,
    tamaño_leyenda    = 8.25,
    vspacio           =-0.45,
    hspacio           =0,
    borde             = 0.08
)

In [0]:
caldri, rc_var = Obj_Seg.CalVar(
  # Parametros de figura individual
  query_uso          = f"CODMES >= 202207 AND CODMES <= 202307",
  pd_seg             = 'PD_APP_HIP_TRDPYME_2Q24',
  rd_aux             = 'RD6',
  pd_comparacion     = False,
  factores_cuentas   = fc,
  factores_montos    = fm,
  mat_importancia    = 0,
  cast_int           = True,
  dim_grafico        = (25, 6),
  pos_leyenda        = (0.5, -0.25),
  punt_mora          = 50,
  proys_def_temp     = 6,
 
  # Parametros de figura final
  tipo_vista         = 'M',
  ncolumnas          = 2,
  nfilas             = 1,
  variables          = ['LTV_SUITE_Q', 'LTV_SALESFORCE_Q'],
  dim_grafico_total  = (20, 5),
  ncol_leyenda       = 5,
  etiquetas_total    = True, 
  pos_etiqueta       = 0.05,
  vspacio            = -0.5,
  hspacio            = 2,
  iv                 = True,
  filtro_var         = "NOT LIKE '%ssing%'"  
)

## Zona de Riesgo de inmueble (*)

En determinado momento se reportó que los datos de esta variable habían cambiado con respecto a lo que originalemente fue modelado. En este apartado se efectua la corrección

In [0]:
altos = [
      "ATE-LIMA", 
      "CARABAYLLO-LIMA", 
      "EL AGUSTINO-LIMA", 
      "LURIN-LIMA", 
      "PACHACAMAC-LIMA",
      "PUENTE PIEDRA-LIMA", 
      "SAN JUAN LURIGANCHO",
      "SAN JUAN MIRAFLORES", 
      "VILLA EL SALVADOR", 
      "VILLA MARIA TRIUNFO", 
      "ATE-LIMA", 
      "CARABAYLLO-LIMA"
]

# Agregar la columna NFT_RIESGO_ZONA con las condiciones especificadas
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
      "NFT_RIESGO_ZONA_ST", 
      when((col("NFT_departamentoinmueble") == "CALLAO") & (col("NFT_distritoinmueble") == "VENTANILLA-LIMA"), "ALTO")
      .when((col("NFT_departamentoinmueble") == "LIMA") & (col("NFT_distritoinmueble").isin(altos)), "ALTO")
      .when((col("NFT_departamentoinmueble").isNull()) | (col("NFT_departamentoinmueble") == "") | (col("NFT_distritoinmueble").isNull()) | (col("NFT_distritoinmueble") == ""), "ND")
      .otherwise("BAJO")
)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "RIESGO_ZONA_ST_DEF",
    when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "ATE-LIMA"), "ALTO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "BARRANCO-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "BELLAVISTA-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "BRE�A-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "CARABAYLLO-LIMA"), "ALTO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "LIMA CERCADO-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "CHACLACAYO-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "CHORRILLOS-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "COMAS-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "EL AGUSTINO-LIMA-LIMA"), "ALTO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "INDEPENDENCIA-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "JESUS MARIA-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "LA MOLINA-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "LA VICTORIA-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "LINCE-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "LOS OLIVOS-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "LURIGANCHO-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "LURIN-LIMA"), "ALTO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "MAGDALENA DEL MAR-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "MIRAFLORES-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "PACHACAMAC-LIMA"), "ALTO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "PUEBLO LIBRE-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "PUENTE PIEDRA-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "RIMAC-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "SAN BORJA-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "SAN ISIDRO-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "SAN JUAN DE LURIGANCHO"), "ALTO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "SAN JUAN LURIGANCHO"), "ALTO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "SAN JUAN MIRAFLORES"), "ALTO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "SAN LUIS'-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "SAN MARTIN DE PORRES"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "SANTA ANITA-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "SURCO-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "SURQUILLO-LIMA"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "VILLA EL SALVADOR"), "ALTO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "VILLA MARIA TRIUNFO"), "ALTO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "MAGDALENA DEL MAR"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "SANTIAGO DE SURCO"), "BAJO")
    .when((col("ESTADOS_DEPARTAMENTO") == "LIMA") & (col("ESTADOS_DISTRITO") == "SAN MIGUEL-LIMA"), "BAJO")
    .otherwise(col("RIESGO_ZONA_ST"))
)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("DEF_NFT_RIESGO_ZONA_ST_ST", when(col("CODMES")>= 202308, col("NFT_RIESGO_ZONA_ST")).otherwise(col("RIESGO_ZONA_ST_DEF")))

driver = 'DEF_NFT_RIESGO_ZONA_ST_ST'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

## Grupo Plazo (*)

Esta variable se encuentra mal trameada en la puntuación y en la construcción de seguimiento. No deben existir plazos mayores a 25 años. En este apartado se efectua la correción

In [0]:
# Cortamos la cantidad de plazo aprobaso
plazo = Obj_Seg.CutVar(variables=['CTDPLAZOAPROBADO', 'NFT_PLAZO'], num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")
print('--- Se verfica que no hay plazos mayores a 25 años ---')

# Cortando la variabe definitiva de plazo
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("DEF_CTDPLAZOAPROBADO", when(col("CODMES")>= 202308, col("NFT_PLAZO")).otherwise(col("CTDPLAZOAPROBADO")))
plazo = Obj_Seg.CutVar(variables=['DEF_CTDPLAZOAPROBADO'], num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

In [0]:
driver = 'DEF_CTDPLAZOAPROBADO_Q'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

# Modulo demográfico

## Antiguedad Laboral (*)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("SUITE_AntLaboralMeses_J", when(col("codmes") < 202307, col("ANT_LABORAL_MESES").cast("double")).otherwise(col("SUITE_AntLaboralMeses").cast("double")))

# Cortamos las variables
ant_laboral = Obj_Seg.CutVar(variables=['SUITE_AntLaboralMeses_J'], num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

In [0]:
driver = 'SUITE_AntLaboralMeses_J_Q'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

## Categoria de Zona (*)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("CAT_ZONA", 
                   when(col("CAT_ZONA1").isNull(), "99. Missing")
                   .when(col("CAT_ZONA1").contains("LIMA"), "Lima")
                   .otherwise("Provincia"))

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "SUITE_CAT_ZONA_J", 
    when(col("codmes") <= 202307, col("CAT_ZONA")).otherwise(col("SUITE_CAT_ZONA"))
)

In [0]:
driver = 'SUITE_CAT_ZONA_J'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

## Actividad Económica del empleador (*)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("DEF_CIIU_DEF", when(col("CODMES")>=202307, col("NFT_CIIU_DEF")).otherwise(col("ANT_CIIU_DEF")))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("DEF_ID_CIIU", when(col("CODMES")>=202307, col("NFT_ID_CIIU")).otherwise(col("ANT_ID_CIIU")))

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("SECTOR_ECONOMICO",
    when(col("DEF_CIIU_DEF").isin(1, 2, 3), "PRIMARIO")
    .when(col("DEF_CIIU_DEF").isin(4, 5, 6), "SECUNDARIO")
    .when(col("DEF_CIIU_DEF").isin(7, 8, 9, 10, 11, 12, 13), "TERCIARIO")
)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("SECTOR_ECONOMICO",
    when(col("NFT_CIIU_DEF").isin(1, 2, 3), "PRIMARIO")
    .when(col("NFT_CIIU_DEF").isin(4, 5, 6), "SECUNDARIO")
    .when(col("NFT_CIIU_DEF").isin(7, 8, 9, 10, 11, 12, 13), "TERCIARIO")
)

In [0]:
driver = 'SECTOR_ECONOMICO'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

In [0]:
driver = 'NFT_ID_CIIU'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (30,7),
  pos_leyenda        = (0.5, -0.4),
  borde              = 0.08
)

In [0]:
caldri_veh_app, rc_dri, rm_dri = Obj_Seg.CalDri(
  driver             = 'NFT_ID_CIIU',
  query_uso          = f"CODMES >= 202110 AND CODMES <= 202304",
  pd_seg             = 'PD_APP_HIP_TRDPYME_2Q24',
  rd_aux             = 'RD12',
  pd_comparacion     = False,
  factores_cuentas   = fc,
  factores_montos    = fm,
  mat_importancia    = 5,
  cast_int           = True, 
  proys_def_temp     = 3,
  titulo             = f'NFT_ID_CIIU',
  dim_grafico        = (35, 6.5),
  etiquetas          = True,
  pos_etiquetas_xc   = [10, 10, 10, 10, 10, 10, 10, 10, 10],
  pos_etiquetas_xm   = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
  pos_leyenda        = (0.5, -0.2),
  punt_mora          = 100,
  iv = True
)

## Edad

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "SUITE_Edad_J", 
    when(col("codmes") <= 202307, col("EDAD")).otherwise(col("SUITE_Edad"))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "SUITE_Edad_J", 
    col('SUITE_Edad_J').cast('double')
)

# Cortamos las variables
ant_laboral = Obj_Seg.CutVar(variables=['SUITE_Edad_J'], num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

In [0]:
driver = 'SUITE_Edad_J_Q'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.3),
  borde              = 0.08
)

## Estado civil (*)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "SUITE_Estcivil_J", 
    when(col("codmes") <= 202307, col("EST_CIV")).otherwise(col("SUITE_Estcivil"))
)

In [0]:
driver = 'SUITE_Estcivil_J'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.3),
  borde              = 0.08
)

## Cluster Vehicular

In [0]:
# Cortamos las variables
ant_laboral = Obj_Seg.CutVar(variables=['CL_VEH_MEAN_st'], num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

In [0]:
from pyspark.sql.functions import exp

# Revertimos la transformación de ODD a PD:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn( 
  "CLU_RIESGO_VEH_PD", 
  1 / (1 + exp(-col("CL_VEH_MEAN_ST")))
)

# Hacemos los cortes de PD Cluster para agrupar los distintos cluster
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
  "CLU_RIESGO_VEH",
  when(col("CLU_RIESGO_VEH_PD") ==	0.45289041446465306, '5. Cluster <=.')
  .when(col("CLU_RIESGO_VEH_PD") ==	0.4528904144528834, '5. Cluster <=.')
  .when(col("CLU_RIESGO_VEH_PD") < 	0.2252013058, '1. Cluster <=4')
  .when(col("CLU_RIESGO_VEH_PD") < 	0.3114250996, '2. Cluster <=6')
  .when(col("CLU_RIESGO_VEH_PD") < 	0.4668103722, '3. Cluster <=8')
  .when(col("CLU_RIESGO_VEH_PD") < 	0.5282556825, '4. Cluster <=9')
)


In [0]:
driver = 'CLU_RIESGO_VEH'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.3),
  borde              = 0.08
)

## Número de hijos

In [0]:
# Cortamos las variables
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "NUMHIJOS1",
    col("NUMHIJOS1").cast('int')
)
ant_laboral = Obj_Seg.CutVar(variables=['NUMHIJOS1'], num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

In [0]:
driver = 'NUMHIJOS1_Q'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.3),
  borde              = 0.08
)

## Ingreso cascada

In [0]:
ant_laboral = Obj_Seg.CutVar(variables=['NFT_IngresoTitular'], num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "DEF_ING_TIT", 
    when(col("codmes") <= 202307, col("ESTADOS_INGRESO_TITULAR")).otherwise(col("NFT_IngresoTitular"))
)
ant_laboral = Obj_Seg.CutVar(variables=['DEF_ING_TIT'], num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

In [0]:
driver = 'DEF_ING_TIT_Q'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.3),
  borde              = 0.08
)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "DEF_ING_CONY", 
    when(col("codmes") <= 202307, col("ESTADOS_INGRESO_CONYUGE")).otherwise(col("NFT_IngresoConyuge"))
)
ant_laboral = Obj_Seg.CutVar(variables=['DEF_ING_CONY'], num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

In [0]:
driver = 'DEF_ING_CONY_Q'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.3),
  borde              = 0.08
)

# Repuntuación

## Modulo Hipotecario

In [0]:

from pyspark.sql.functions import log10

dummy_list = [1111111111, -1111111111, 2222222222, -2222222222, 3333333333, -3333333333, 4444444444, 5555555555, 6666666666, 7777777777, 99999]

#--------------------------------#
#-------- LTV SALESFORCE --------#
#--------------------------------#

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "LTV_1_REP_SALESFORCE",
    when(
        (col("LTV_CORREGIDO_VF") < 0.64288253401604) & (~col("LTV_CORREGIDO_VF").isin(dummy_list)),
        lit(0.64288253401604)
    ).when(
        (col("LTV_CORREGIDO_VF") > 0.95) & (~col("LTV_CORREGIDO_VF").isin(dummy_list)),
        lit(0.95)
    ).otherwise(col("LTV_CORREGIDO_VF"))
)

threshold = 0.66449996056493
multiplier = 7.42270169852774
offset = -3.5025279917365
default_value = -4.82009495731347

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "LTV_1_D2_REP_SALESFORCE",
    when(~col("LTV_1_REP_SALESFORCE").isin(dummy_list),
    when(col("LTV_1_REP_SALESFORCE") >= threshold, log10(col("LTV_1_REP_SALESFORCE")) * multiplier + offset).otherwise(default_value))
    .otherwise(None)
)

# Constantes
default_value = -4.71505889542265
upper_limit = 5.30330490805907
lower_limit = -5.30330490805907

# Aplicar condiciones
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "LTV_1_D2_REP_SALESFORCE",
    when(col("LTV_1_REP_SALESFORCE").isNull(), default_value)  # Si LTV_1 es nulo
    .otherwise(col("LTV_1_D2_REP_SALESFORCE"))  # Mantener el valor actual si no es nulo
)

# Limitar el rango de LTV_1_D2
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "LTV_1_D2_REP__SALESFORCE",
    when(col("LTV_1_D2_REP_SALESFORCE") > upper_limit, upper_limit)
    .when(col("LTV_1_D2_REP_SALESFORCE") < lower_limit, lower_limit)
    .otherwise(col("LTV_1_D2_REP_SALESFORCE"))
)


#---------------------------#
#-------- LTV SUITE --------#
#---------------------------#

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "LTV_1_REP_SUITE",
    when(
        (col("LTV_SUITE") < 0.64288253401604) & (~col("LTV_SUITE").isin(dummy_list)),
        lit(0.64288253401604)
    ).when(
        (col("LTV_SUITE") > 0.95) & (~col("LTV_SUITE").isin(dummy_list)),
        lit(0.95)
    ).otherwise(col("LTV_SUITE"))
)

threshold = 0.66449996056493
multiplier = 7.42270169852774
offset = -3.5025279917365
default_value = -4.82009495731347

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "LTV_1_D2_REP_SUITE",
    when(~col("LTV_1_REP_SUITE").isin(dummy_list),
    when(col("LTV_1_REP_SUITE") >= threshold, log10(col("LTV_1_REP_SUITE")) * multiplier + offset).otherwise(default_value))
    .otherwise(None)
)

# Constantes
default_value = -4.71505889542265
upper_limit = 5.30330490805907
lower_limit = -5.30330490805907

# Aplicar condiciones
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "LTV_1_D2_REP_SUITE",
    when(col("LTV_1_REP_SUITE").isNull(), default_value)  # Si LTV_1 es nulo
    .otherwise(col("LTV_1_D2_REP_SUITE"))  # Mantener el valor actual si no es nulo
)

# Limitar el rango de LTV_1_D2
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "LTV_1_D2_REP_SUITE",
    when(col("LTV_1_D2_REP_SUITE") > upper_limit, upper_limit)
    .when(col("LTV_1_D2_REP_SUITE") < lower_limit, lower_limit)
    .otherwise(col("LTV_1_D2_REP_SUITE"))
)

#-----------------------------#
#-------- RIESGO ZONA --------#
#-----------------------------#


Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "RIESGO_ZONA_ST_ST_REP",
    when(col("DEF_NFT_RIESGO_ZONA_ST_ST") == "ALTO", -4.62497281328427)
    .when(col("DEF_NFT_RIESGO_ZONA_ST_ST") == "BAJO", -5.21697172709408)
    .when(col("DEF_NFT_RIESGO_ZONA_ST_ST") == "ND", -4.61470207761345)
    .otherwise(None)  # Puedes cambiar esto si quieres un valor por defecto distinto
)

#--------------------------------#
#-------- PLAZO APROBADO --------#
#--------------------------------#

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "GRUPO_PLAZO_REP",
    when((col("DEF_CTDPLAZOAPROBADO") >= 1) & (col("DEF_CTDPLAZOAPROBADO") < 5), 1)
    .when((col("DEF_CTDPLAZOAPROBADO") >= 5) & (col("DEF_CTDPLAZOAPROBADO") < 10), 2)
    .when((col("DEF_CTDPLAZOAPROBADO") >= 10) & (col("DEF_CTDPLAZOAPROBADO") < 15), 3)
    .when((col("DEF_CTDPLAZOAPROBADO") >= 15) & (col("DEF_CTDPLAZOAPROBADO") < 20), 4)
    .when((col("DEF_CTDPLAZOAPROBADO") >= 20) & (col("DEF_CTDPLAZOAPROBADO") < 25), 5)
    .when((col("DEF_CTDPLAZOAPROBADO") >= 25) & (col("DEF_CTDPLAZOAPROBADO") < 30), 6)
    .when((col("DEF_CTDPLAZOAPROBADO") >= 30) & (col("DEF_CTDPLAZOAPROBADO") < 35), 7)
    .when((col("DEF_CTDPLAZOAPROBADO") >= 35) | (col("DEF_CTDPLAZOAPROBADO").isNull()), 8)
    .otherwise(None)
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "GRUPO_PLAZO_st_REP",
    when(col("GRUPO_PLAZO_REP") == 1, -6.0734)
    .when(col("GRUPO_PLAZO_REP") == 2, -5.7347)
    .when(col("GRUPO_PLAZO_REP") == 3, -5.396)
    .when(col("GRUPO_PLAZO_REP") == 4, -4.89429858043284)
    .when(col("GRUPO_PLAZO_REP") == 5, -4.62790967295758)
    .when(col("GRUPO_PLAZO_REP") == 6, -4.3727734291934)
    .when(col("GRUPO_PLAZO_REP") == 7, -4.0674)
    .when(col("GRUPO_PLAZO_REP") == 8, -3.8159)
    .otherwise(None)  # Puedes ajustar esto si deseas un valor por defecto distinto
)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "XB_HIP_REP_SALESFORCE",
    16.1771
    + 1.4716 * col("LTV_1_D2_REP_SALESFORCE")
    + 0.9372 * col("GRUPO_PLAZO_st_REP")
    + 1.1548 * col("RIESGO_ZONA_ST_ST_REP")
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "XB_HIP_REP_SUITE",
    16.1771
    + 1.4716 * col("LTV_1_D2_REP_SUITE")
    + 0.9372 * col("GRUPO_PLAZO_st_REP")
    + 1.1548 * col("RIESGO_ZONA_ST_ST_REP")
)

### Constraste de discriminación

In [0]:
var_pd = ["XB_HIP_REP_SALESFORCE", "XB_HIP_REP_SUITE", "XB_HIP"]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro = '',
  codmes_ini   = 202206,
  codmes_fin   = 202401,
  amplitud     = 24,
  disc_target  = [24, 21,18],
  umbral_sup   = 0.4,
  umbral_inf   = 0.3,
  moviles      = True,
  dim_grafico  = (22.5, 6),
  pos_leyenda  = (0.5, -0.35),
  etiquetas    = True,
  gini_abs     = False,
 
  # Parametros de figura final
  ncolumnas = 3,
  nfilas    = 1,
  variables = var_pd,
  dim_grafico_total = (25, 6),
  ncol_leyenda = 5,
  vspacio=-0.25,
  hspacio=4,
  tamaño_etiqueta = 14 
)

### Constraste de estabilidad

In [0]:
var_pd_q = Obj_Seg.CutVar(variables=var_pd, num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

In [0]:
psi_dri = Obj_Seg.MixPSIVar(
  # Parametros de figura individual
    query_uso         ='CODMES >= 202011',
    cast_int          =True,
    codmes_inicio1    =202011,
    codmes_fin1       =202204,
    codmes_inicio2    =202501,
    codmes_fin2       =202504,
    dim_grafico       =(25, 6),
    pos_leyenda       =(0.5, -0.25),
 
    # Parametros de figura final
    tipo_vista        = 'M',
    ncolumnas         = 3,
    nfilas            = 1,
    variables         = var_pd_q,
    dim_grafico_total = (25, 6),
    ncol_leyenda      =5,
    tamaño_leyenda    = 8.25,
    vspacio           =-0.45,
    hspacio           =0,
    borde             = 0.08
)

## Modulo demográfico

In [0]:
#------------------------------------#
#-------- ANTIGUEDAD LABORAL --------#
#------------------------------------#

from pyspark.sql.functions import col, when, log10, lit

# Paso 1: Reemplazar valores extremos si no están en dummy_list
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "SUITE_AntLaboralMeses_J",
    when((~col("SUITE_AntLaboralMeses_J").isin(dummy_list)) & (col("SUITE_AntLaboralMeses_J") < 1), 1)
    .when((~col("SUITE_AntLaboralMeses_J").isin(dummy_list)) & (col("SUITE_AntLaboralMeses_J") > 72), 72)
    .otherwise(col("SUITE_AntLaboralMeses_J"))
)

# Paso 2: Calcular ANT_LABORAL_MESES_E2 según condiciones
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "ANT_LABORAL_MESES_E2_REP",
    when(col("SUITE_AntLaboralMeses_J").isNull(), -0.3120167161602)
    .when(~col("SUITE_AntLaboralMeses_J").isin(dummy_list),
          when(col("SUITE_AntLaboralMeses_J") >= 1.9189008504925,
               log10(col("SUITE_AntLaboralMeses_J")) ** 2 * (-0.44887796667188)
               + log10(col("SUITE_AntLaboralMeses_J")) * 0.25411209305111
               + 0.37338265275158)
          .otherwise(0.40934618885114))
    .otherwise(None)
)

# Paso 3: Limitar valores extremos de ANT_LABORAL_MESES_E2
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "ANT_LABORAL_MESES_E2_REP",
    when(col("ANT_LABORAL_MESES_E2_REP") > 5.30330490805907, 5.30330490805907)
    .when(col("ANT_LABORAL_MESES_E2_REP") < -5.30330490805907, -5.30330490805907)
    .otherwise(col("ANT_LABORAL_MESES_E2_REP"))
)

#--------------------------------#
#-------- CATEGORIA ZONA --------#
#--------------------------------#

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("DEF_DESDITRITO", when(col("codmes") <= 202307, col("DESCODDISTRITO")).otherwise(col("SUITE_DESDISTRITO")))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("DEF_DESPROVINCIA", when(col("codmes") <= 202307, col("DESCODPROVINCIA")).otherwise(col("SUITE_DESPROVINCIA")))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("DEF_DESDEPARTAMENTO", when(col("codmes") <= 202307, col("DESCODDEPARTAMENTO")).otherwise(col("SUITE_desdepartamento")))


Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "CAT_ZONA1_REP",
    when(
        (col("DEF_DESDEPARTAMENTO").isNotNull()) & (col("DEF_DESDEPARTAMENTO") != "") & (col("DEF_DESDEPARTAMENTO") != "LIMA"),
        when(col("DEF_DESDEPARTAMENTO") == "CERRO DE PASCO", "CERRO_DE_PASCO")
        .when(col("DEF_DESDEPARTAMENTO") == "MADRE DE DIOS", "MADRE_DE_DIOS")
        .when(col("DEF_DESDEPARTAMENTO") == "SAN MARTIN", "SAN_MARTIN")
        .when((col("DEF_DESDEPARTAMENTO") == "LA LIBERTAD") | (col("DEF_DESDEPARTAMENTO") == "LALIBERTAD"), "LA_LIBERTAD")
        .otherwise(col("DEF_DESDEPARTAMENTO"))
    )
)

# Diccionario de distritos a zonas
distrito_zona = {
    "ANCON": "LIMA_NORTE",
    "ATE": "LIMA_ESTE",
    "CIENEGUILLA": "LIMA_ESTE",
    "BARRANCO": "LIMA_SUR",
    "BREÑA": "LIMA_CENTRO",
    "CARABAYLLO": "LIMA_NORTE",
    "CHACLACAYO": "LIMA_ESTE",
    "CHORRILLOS": "LIMA_SUR",
    "COMAS": "LIMA_NORTE",
    "EL AGUSTINO": "LIMA_ESTE",
    "INDEPENDENCIA": "LIMA_NORTE",
    "JESUS MARIA": "LIMA_MODERNA",
    "LA MOLINA": "LIMA_MODERNA",
    "LA VICTORIA": "LIMA_CENTRO",
    "LIMA CERCADO": "LIMA_CENTRO",
    "LINCE": "LIMA_MODERNA",
    "LOS OLIVOS": "LIMA_NORTE",
    "LURIGANCHO": "LIMA_ESTE",
    "LURIN": "LIMA_SUR",
    "MAGDALENA DEL MAR": "LIMA_MODERNA",
    "MIRAFLORES": "LIMA_MODERNA",
    "PACHACAMAC": "LIMA_SUR",
    "PUCUSANA": "LIMA_SUR",
    "PUEBLO LIBRE": "LIMA_MODERNA",
    "PUENTE PIEDRA": "LIMA_NORTE",
    "PUNTA HERMOSA": "LIMA_SUR",
    "PUNTA NEGRA": "LIMA_SUR",
    "RIMAC": "LIMA_CENTRO",
    "SAN BARTOLO": "LIMA_SUR",
    "SAN BORJA": "LIMA_MODERNA",
    "SAN ISIDRO": "LIMA_MODERNA",
    "SAN JUAN LURIGANCHO": "LIMA_ESTE",
    "SAN JUAN MIRAFLORES": "LIMA_SUR",
    "SAN LUIS": "LIMA_CENTRO",
    "SAN MARTIN DE PORRES": "LIMA_NORTE",
    "SAN MIGUEL": "LIMA_MODERNA",
    "SANTA ANITA": "LIMA_ESTE",
    "SANTA MARIA DEL MAR": "LIMA_SUR",
    "SANTA ROSA": "LIMA_NORTE",
    "SANTIAGO DE SURCO": "LIMA_MODERNA",
    "SURQUILLO": "LIMA_MODERNA",
    "VILLA EL SALVADOR": "LIMA_SUR",
    "VILLA MARIA TRIUNFO": "LIMA_SUR"
}

from pyspark.sql.functions import create_map, lit
from itertools import chain

# Convertir el diccionario a un mapa para PySpark
mapping_expr = create_map([lit(x) for x in chain(*distrito_zona.items())])

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "CAT_ZONA1_REP",
    when(
        (col("DEF_DESDEPARTAMENTO") == "LIMA") & (col("DEF_DESPROVINCIA") == "LIMA"),
        mapping_expr.getItem(col("DEF_DESDITRITO"))
    ).when(
        (col("DEF_DESDEPARTAMENTO") == "LIMA") & (col("DEF_DESPROVINCIA") != "LIMA"),
        lit("LIMA_PROV")
    ).otherwise(col("CAT_ZONA1_REP"))  # Mantener lo que ya se asignó antes
)

from pyspark.sql.functions import create_map, lit, col, when
from itertools import chain

zona_coef = {
    'AMAZONAS': -0.039938704,
    'ANCASH': -0.039938704,
    'APURIMAC': 0.0758135367,
    'AREQUIPA': -0.617007477,
    'AYACUCHO': 0.0333518574312501,
    'CAJAMARCA': -0.282994571,
    'CALLAO': 0.0758135367,
    'CERRO_DE_PASCO': -0.484944634,
    'CUZCO': -0.484944634,
    'HUANCAVELICA': 0.0333518574312501,
    'HUANUCO': -0.135237963,
    'ICA': -0.039938704,
    'JUNIN': -0.282994571,
    'LAMBAYEQUE': -0.135237963,
    'LA_LIBERTAD': -0.135237963,
    'LIMA_CENTRO': -0.135237963,
    'LIMA_ESTE': -0.135237963,
    'LIMA_MODERNA': -0.741004134,
    'LIMA_NORTE': -0.039938704,
    'LIMA_PROV': -0.135237963,
    'LIMA_SUR': -0.135237963,
    'LORETO': 0.385320542,
    'MADRE_DE_DIOS': 0.0758135367,
    'MOQUEGUA': -0.617007477,
    'PIURA': 0.0333518574312501,
    'PUNO': 0.0333518574312501,
    'SAN_MARTIN': -0.135237963,
    'TACNA': -0.484944634,
    'TUMBES': 0.0333518574312501,
    'UCAYALI': 0.2909809729,
    'AMERICA': -0.128832872,
    'ASIA': 0.193371393,
    'EUROPA': -0.223441042016937,
    'OCEANIA': 0.385320542,
    '': -0.101201527,
    '-': -0.101201527
}

mapping_expr = create_map([lit(x) for x in chain(*zona_coef.items())])

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "CAT_ZONA1_C_st_REP",
    mapping_expr.getItem(col("CAT_ZONA1_REP"))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "CAT_ZONA1_C_st_REP",
    when(col("CAT_ZONA1_C_st_REP").isNull(), -0.101201527).otherwise(col("CAT_ZONA1_C_st_REP"))
)

#-------------------------------------#
#-------- ACTIVIDAD ECONOMICA --------#
#-------------------------------------#

from pyspark.sql.functions import create_map, lit, col
from itertools import chain

ciiu_map = {
    1: -0.1801052251765060,
    2: -0.0569214964619532,
    3: -0.8740183129348840,
    4: -0.2947593978359910,
    5: -0.8740183129348840,
    6: -0.0569214964619532,
    7: -0.2947593978359910,
    8: -0.1801052251765060,
    9: -0.4582102012890810,
    10: -0.8740183129348840,
    11: -0.2947593978359910,
    12: -0.4582102012890810,
    13: -0.0569214964619532
}
default_value = -0.0478452553157524

mapping_expr = create_map([lit(x) for x in chain(*ciiu_map.items())])

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "CL_CIIU_ST_REP",
    mapping_expr.getItem(col("DEF_CIIU_DEF"))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "CL_CIIU_ST_REP",
    when(col("CL_CIIU_ST_REP").isNull(), default_value).otherwise(col("CL_CIIU_ST_REP"))
)

#------------------------------#
#-------- ESTADO CIVIL --------#
#------------------------------#

from pyspark.sql.functions import create_map, lit, col, when
from itertools import chain

estado_civil_map = {
    'CAS': -2.71052690690738,
    'CBS': -1.3465386059661,
    'CON': -1.01448118381917,
    'SEP': -1.71018184183643,
    'DIV': -1.71018184183643,
    'SOL': -1.31089975256589,
    'VIU': -1.91397294136183
}
default_value = -1.68975654862259

mapping_expr = create_map([lit(x) for x in chain(*estado_civil_map.items())])

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "EST_CIV_st_REP",
    mapping_expr.getItem(col("SUITE_Estcivil_J"))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "EST_CIV_st_REP",
    when(col("EST_CIV_st_REP").isNull(), default_value).otherwise(col("EST_CIV_st_REP"))
)

#---------------------------------#
#-------- INGRESO CASCADA --------#
#---------------------------------#

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "ING_SOL_REP",
    col('DEF_ING_CONY') + col('DEF_ING_TIT')
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("ING_CASCADA2_REP", col("ING_SOL_REP")) \
    .withColumn("ING_CASCADA2_F", col("ING_SOL_REP"))\
    .withColumn("ING_CASCADA3", col("ING_SOL_REP"))\
    .withColumn("ING_CASCADA3_F", col("ING_SOL_REP"))

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "ING_CASCADA3_F",
    when((~col("ING_CASCADA3_F").isin(dummy_list)) & (col("ING_CASCADA3_F") < 733.42), 733.42)
    .when((~col("ING_CASCADA3_F").isin(dummy_list)) & (col("ING_CASCADA3_F") > 11466.6615), 11466.6615)
    .otherwise(col("ING_CASCADA3_F"))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "ING_CASCADA2_D2_REP",
    when(col("ING_CASCADA2_F").isNull(), -0.37028113613139)
    .when(~col("ING_CASCADA2_F").isin(dummy_list),
          when(col("ING_CASCADA2_F") >= 772.712680373535,
               log10(col("ING_CASCADA2_F")) * (-1.5728915038144) + 4.86125959214205)
          .otherwise(0.31872055547668))
    .otherwise(None)
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "ING_CASCADA3_D2_REP",
    when(col("ING_CASCADA3_F").isNull(), -0.37028113613139)
    .when(~col("ING_CASCADA3_F").isin(dummy_list),
          when(col("ING_CASCADA3_F") >= 772.712680373535,
               log10(col("ING_CASCADA3_F")) * (-1.5728915038144) + 4.86125959214205)
          .otherwise(0.31872055547668))
    .otherwise(None)
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "ING_CASCADA2_D2_REP",
    when(col("ING_CASCADA2_D2_REP") > 5.30330490805907, 5.30330490805907)
    .when(col("ING_CASCADA2_D2_REP") < -5.30330490805907, -5.30330490805907)
    .otherwise(col("ING_CASCADA2_D2_REP"))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "ING_CASCADA3_D2_REP",
    when(col("ING_CASCADA3_D2_REP") > 5.30330490805907, 5.30330490805907)
    .when(col("ING_CASCADA3_D2_REP") < -5.30330490805907, -5.30330490805907)
    .otherwise(col("ING_CASCADA3_D2_REP"))
)


In [0]:
from pyspark.sql.functions import col

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "XB_DEM_REP",
    2.9043
    + 0.4919 * col("ANT_LABORAL_MESES_E2_REP")
    + 0.6787 * col("CAT_ZONA1_C_st_REP")
    + 0.8429 * col("CL_CIIU_st_REP")
    + 0.6557 * col("CL_VEH_MEAN_st")
    + 0.3951 * col("EDAD_E1")
    + 0.1797 * col("EST_CIV_st_REP")
    + 0.5159 * col("ING_CASCADA2_D2_REP")
    + 1.0291 * col("NUMHIJOS1_C2_st")
)

### Contraste de discriminación

In [0]:
var_pd = ["XB_DEM_REP", "XB_DEM"]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro = '',
  codmes_ini   = 202206,
  codmes_fin   = 202312,
  amplitud     = 24,
  disc_target  = [24, 21,18],
  umbral_sup   = 0.4,
  umbral_inf   = 0.3,
  moviles      = True,
  dim_grafico  = (22.5, 6),
  pos_leyenda  = (0.5, -0.35),
  etiquetas    = True,
  gini_abs     = False,
 
  # Parametros de figura final
  ncolumnas = 2,
  nfilas    = 1,
  variables = var_pd,
  dim_grafico_total = (20, 6),
  ncol_leyenda = 5,
  vspacio=-0.25,
  hspacio=4,
  tamaño_etiqueta = 14 
)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "XB_DEM_S_ADCW_REP",
    0.2738 + 0.4184 * col('XB_DEM_REP')
)

In [0]:
var_pd = ["XB_DEM_S_ADCW_REP", "XB_DEM_S_ADCW"]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro = '',
  codmes_ini   = 202206,
  codmes_fin   = 202312,
  amplitud     = 24,
  disc_target  = [24, 21,18],
  umbral_sup   = 0.4,
  umbral_inf   = 0.3,
  moviles      = True,
  dim_grafico  = (22.5, 6),
  pos_leyenda  = (0.5, -0.35),
  etiquetas    = True,
  gini_abs     = False,
 
  # Parametros de figura final
  ncolumnas = 2,
  nfilas    = 1,
  variables = var_pd,
  dim_grafico_total = (20, 6),
  ncol_leyenda = 5,
  vspacio=-0.25,
  hspacio=4,
  tamaño_etiqueta = 14 
)

### Constraste de estabilidad

In [0]:
var_pd_q = Obj_Seg.CutVar(variables=var_pd, num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

In [0]:
psi_dri = Obj_Seg.MixPSIVar(
  # Parametros de figura individual
    query_uso         ='CODMES >= 202011',
    cast_int          =True,
    codmes_inicio1    =202011,
    codmes_fin1       =202204,
    codmes_inicio2    =202501,
    codmes_fin2       =202504,
    dim_grafico       =(25, 6),
    pos_leyenda       =(0.5, -0.25),
 
    # Parametros de figura final
    tipo_vista        = 'M',
    ncolumnas         = 2,
    nfilas            = 1,
    variables         = var_pd_q,
    dim_grafico_total = (17, 6),
    ncol_leyenda      =5,
    tamaño_leyenda    = 8.25,
    vspacio           =-0.45,
    hspacio           =0,
    borde             = 0.08
)

## Puntuación producto

In [0]:
# XB modulo clásicos
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'XB_AD_HOC_REP',
    0.2 * col('XB_ACT_S_ADCW') + 
    0.2 * col('XB_PAS_S_ADCW') +
    0.2 * col('XB_TRX_S_ADCW') + 
    0.2 * col('XB_RCC_S_ADCW') + 
    0.2 * col('XB_DEM_S_ADCW_REP')
)

# XB troncal
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'XB_ADC_POND85_REP_SALESFORCE',
    0.85 * col('XB_AD_HOC_REP') + 
    0.15 * col('XB_HIP_REP_SALESFORCE')
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'XB_ADC_POND85_REP_SUITE',
    0.85 * col('XB_AD_HOC_REP') + 
    0.15 * col('XB_HIP_REP_SUITE')
)

# input para puntuar las logicas de ISA


## Re-Puntuación modelo deprecado

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("FLG_SOLTERO_ESPJ", col("FLG_SOLTERO_ESPJ").cast('double'))

# Calibrado deprecado
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'XB_P7_REP',
    0.7643010711 + 
    3.486179056*col('XB_ADC_POND85_REP_SALESFORCE') + 
    0.454204416*col('FLG_SOLTERO_ESPJ') + 
    0.5161483188*col('D_PYME') + 
    0.3950922249*col('SEGMENTO2_st') + 
    0.5719599111*col('FLG_NODEP_ESPJ') + 
    0.6680047673*col('ZONA_PAUTA5_st')
)

# Ajuste de calibrado deprecado
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'XB_P7_AJUS_REP',
    when(col('D_PYME')==0, 0.3357247425 + 1.0090882734*col('XB_P7_REP'))
    .when(col('D_PYME')==1, 0.0187091756 + 1.0007877789*col('XB_P7_REP'))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('PD_P7_ajus_REP', 1/(1+exp(-col('XB_P7_REP'))))

## Re-Puntuación nuevo calibrado

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'RCC_PCT_UTL6_UTL24_RT_U24_1',
    when((col('RCC_PCT_UTL6_UTL24_RT_U24') < 0) & (~col('RCC_PCT_UTL6_UTL24_RT_U24').isin(dummy_list)), 0)
    .when((col('RCC_PCT_UTL6_UTL24_RT_U24') > 4.75750721) & (~col('RCC_PCT_UTL6_UTL24_RT_U24').isin(dummy_list)), 4.75750721)
    .otherwise(col('RCC_PCT_UTL6_UTL24_RT_U24'))
)
    
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'RCC_PCT_UTL6_UTL24_RT_U24_A2',
    when(
        (col('RCC_PCT_UTL6_UTL24_RT_U24_1') >= 1.54656615160457) & (~col('RCC_PCT_UTL6_UTL24_RT_U24_1').isin(dummy_list)), 
        col('RCC_PCT_UTL6_UTL24_RT_U24')*( 0.6065153511141)+( -5.73322836042411)
    )
    .when(
        (col('RCC_PCT_UTL6_UTL24_RT_U24_1') < 1.54656615160457) & (~col('RCC_PCT_UTL6_UTL24_RT_U24_1').isin(dummy_list)), 
        -4.79521224796248
    )
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'RCC_PCT_UTL6_UTL24_RT_U24_A2',
    when(col('RCC_PCT_UTL6_UTL24_RT_U24_1').isNull(), -4.09850257237076)
    .when(col('RCC_PCT_UTL6_UTL24_RT_U24_1') == 1111111111, -3.25809653802148)
    .when(col('RCC_PCT_UTL6_UTL24_RT_U24_1') == 4444444444, -3.15965646520823)
    .when(col('RCC_PCT_UTL6_UTL24_RT_U24_1') == 5555555555, -3.86423234159179)
    .when(col('RCC_PCT_UTL6_UTL24_RT_U24_1') == 6666666666, -3.43398720448514)
    .when(col('RCC_PCT_UTL6_UTL24_RT_U24_A2').isNull(), -4.53576739442981)
    .when(col('RCC_PCT_UTL6_UTL24_RT_U24_A2')>5.30330490805907, 5.30330490805907)
    .when(col('RCC_PCT_UTL6_UTL24_RT_U24_A2')<-5.30330490805907, -5.30330490805907)
    .otherwise(col('RCC_PCT_UTL6_UTL24_RT_U24_A2'))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'EXP_PCT_EVOL_SHIP_U6M_RT_U24_1',
    when((col('EXP_PCT_EVOL_SHIP_U6M_RT_U24') < 0) & (~col('EXP_PCT_EVOL_SHIP_U6M_RT_U24').isin(dummy_list)), -0.23567654)
    .when((col('EXP_PCT_EVOL_SHIP_U6M_RT_U24') > 4) & (~col('EXP_PCT_EVOL_SHIP_U6M_RT_U24').isin(dummy_list)), 4)
    .otherwise(col('EXP_PCT_EVOL_SHIP_U6M_RT_U24'))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'EXP_PCT_EVOL_SHIP_U6M_RT_U24_A3',
    when(
        (col('EXP_PCT_EVOL_SHIP_U6M_RT_U24_1') <= 1.60497414842296) & (~col('EXP_PCT_EVOL_SHIP_U6M_RT_U24_1').isin(dummy_list)), 
        col('EXP_PCT_EVOL_SHIP_U6M_RT_U24_1')*( 0.9920846011259)+( -5.418514499396)
    )
    .when(
        (col('EXP_PCT_EVOL_SHIP_U6M_RT_U24_1') > 1.60497414842296) & (~col('EXP_PCT_EVOL_SHIP_U6M_RT_U24_1').isin(dummy_list)), 
        -3.82624436154042
    )
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'EXP_PCT_EVOL_SHIP_U6M_RT_U24_A3',
    when(col('EXP_PCT_EVOL_SHIP_U6M_RT_U24_1').isNull(), -4.71734613341528 )
    .when(col('EXP_PCT_EVOL_SHIP_U6M_RT_U24_1') == 1111111111, -4.33983962163802)
    .when(col('EXP_PCT_EVOL_SHIP_U6M_RT_U24_1') == -1111111111, -4.44029554279785)
    .when(col('EXP_PCT_EVOL_SHIP_U6M_RT_U24_1') == 4444444444, -3.56953269648137)
    .when(col('EXP_PCT_EVOL_SHIP_U6M_RT_U24_1') == 5555555555, -3.61091791264422)
    .when(col('EXP_PCT_EVOL_SHIP_U6M_RT_U24_1') == 6666666666, -4.33983962163802)
    .when(col('EXP_PCT_EVOL_SHIP_U6M_RT_U24_A3').isNull(), -4.33983962163802)
    .when(col('EXP_PCT_EVOL_SHIP_U6M_RT_U24_A3')>5.30330490805907, 5.30330490805907)
    .when(col('EXP_PCT_EVOL_SHIP_U6M_RT_U24_A3')<-5.30330490805907, -5.30330490805907)
    .otherwise(col('EXP_PCT_EVOL_SHIP_U6M_RT_U24_A3'))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'SLD_PRM_PAS_MIN_24_24_RT_U24_1',
    when((col('SLD_PRM_PAS_MIN_24_24_RT_U24') < 0) & (~col('SLD_PRM_PAS_MIN_24_24_RT_U24').isin(dummy_list)), 0)
    .when((col('SLD_PRM_PAS_MIN_24_24_RT_U24') > 0.99141628) & (~col('SLD_PRM_PAS_MIN_24_24_RT_U24').isin(dummy_list)), 0.99141628)
    .otherwise(col('SLD_PRM_PAS_MIN_24_24_RT_U24'))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'SLD_PRM_PAS_MIN_24_24_RT_U24_I',
    ((3.40467318899109)/(1+exp(-((((col('SLD_PRM_PAS_MIN_24_24_RT_U24')-(0 ))/((0.99141628 )-(0 )))-(9.9621725832108E-9))*(-40.5181321064318)))))+(-5.02922159797982)
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'SLD_PRM_PAS_MIN_24_24_RT_U24_I',
    when(col('SLD_PRM_PAS_MIN_24_24_RT_U24_1').isNull(), -4.55212096489827)
    .when(col('SLD_PRM_PAS_MIN_24_24_RT_U24_1') == 4444444444, -2.79575578152131)
    .when(col('SLD_PRM_PAS_MIN_24_24_RT_U24_1') == 5555555555, -3.85014760171005)
    .when(col('SLD_PRM_PAS_MIN_24_24_RT_U24_I').isNull(), -4.43288505190737)
    .when(col('SLD_PRM_PAS_MIN_24_24_RT_U24_I')> 5.30330490805907, 5.30330490805907)
    .when(col('SLD_PRM_PAS_MIN_24_24_RT_U24_I')< -5.30330490805907 , -5.30330490805907)
    .otherwise(col('SLD_PRM_PAS_MIN_24_24_RT_U24_I'))+ (0.66733158)
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'PD_APP_HIP_TRDPYME_IDIO_REP_SALESFORCE',
    1/(1+exp(-col('XB_APP_HIP_TRDPYME_IDIO_REP_SALESFORCE')))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'XB_APP_HIP_TRDPYME_2Q24_AJUS_REP_SALESFORCE',
    0.3012943064 + 1.0131576471*col('XB_APP_HIP_TRDPYME_IDIO_REP_SALESFORCE')
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('PD_APP_HIP_TRDPYME_2Q24_REP_SALESFORCE', 1/(1+exp(-col('XB_APP_HIP_TRDPYME_2Q24_AJUS_REP_SALESFORCE'))))

# Puntuación Seguimiento con LTV Suite
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'XB_APP_HIP_TRDPYME_IDIO_REP_SUITE',
    + 3.91310351 + 4.02126781*col('XB_ADC_POND85_REP_SUITE') + 0.56543944*col('RCC_PCT_UTL6_UTL24_RT_U24_A2') 
	+ 0.78436330*col('EXP_PCT_EVOL_SHIP_U6M_RT_U24_A3') + 0.37055435*col('SLD_PRM_PAS_MIN_24_24_RT_U24_I') + (0.66733158)
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'PD_APP_HIP_TRDPYME_IDIO_REP_SUITE',
    1/(1+exp(-col('XB_APP_HIP_TRDPYME_IDIO_REP_SUITE')))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'XB_APP_HIP_TRDPYME_2Q24_AJUS_REP_SUITE',
    0.3012943064 + 1.0131576471*col('XB_APP_HIP_TRDPYME_IDIO_REP_SUITE')
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('PD_APP_HIP_TRDPYME_2Q24_REP_SUITE', 1/(1+exp(-col('XB_APP_HIP_TRDPYME_2Q24_AJUS_REP_SUITE'))))

# Puntuación Fábrica
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'XB_APP_HIP_TRDPYME_IDIO_REP_FAB',
    + 3.91310351 + 4.02126781*col('XB_ADC_POND85_REP_SALESFORCE') + 0.56543944*col('UTIL_PROM_6_24_A2') 
	+ 0.78436330*col('DEU_NOHIP_6_24_A3') + 0.37055435*col('PAS_MIN_PROM24_I') + (0.66733158)
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'PD_APP_HIP_TRDPYME_IDIO_REP_FAB',
    1/(1+exp(-col('XB_APP_HIP_TRDPYME_IDIO_REP_FAB')))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'XB_APP_HIP_TRDPYME_2Q24_AJUS_REP_FAB',
    0.3012943064 + 1.0131576471*col('XB_APP_HIP_TRDPYME_IDIO_REP_FAB')
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('PD_APP_HIP_TRDPYME_2Q24_REP_FAB', 1/(1+exp(-col('XB_APP_HIP_TRDPYME_2Q24_AJUS_REP_FAB'))))

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('PD_BASE_ESPEJO', col('PD_BASE_ESPEJO').cast('double'))
var_pd = ["PD_P7_ajus_REP", "PD_BASE_ESPEJO"]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro = '',
  codmes_ini   = 202206,
  codmes_fin   = 202312,
  amplitud     = 24,
  disc_target  = [24, 21,18],
  umbral_sup   = 0.5,
  umbral_inf   = 0.4,
  moviles      = True,
  dim_grafico  = (22.5, 6),
  pos_leyenda  = (0.5, -0.35),
  etiquetas    = True,
  gini_abs     = False,
 
  # Parametros de figura final
  ncolumnas = 2,
  nfilas    = 1,
  variables = var_pd,
  dim_grafico_total = (20, 6),
  ncol_leyenda = 5,
  vspacio=-0.25,
  hspacio=4,
  tamaño_etiqueta = 14 
)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('PD_BASE_ESPEJO', col('PD_BASE_ESPEJO').cast('double'))
var_pd = ["PD_APP_HIP_TRDPYME_2Q24_REP_SALESFORCE", "PD_APP_HIP_TRDPYME_2Q24_REP_SUITE", "XB_APP_HIP_TRDPYME_IDIO_REP_FAB"]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro = '',
  codmes_ini   = 202206,
  codmes_fin   = 202312,
  amplitud     = 24,
  disc_target  = [24, 21,18],
  umbral_sup   = 0.5,
  umbral_inf   = 0.4,
  moviles      = True,
  dim_grafico  = (22.5, 6),
  pos_leyenda  = (0.5, -0.35),
  etiquetas    = True,
  gini_abs     = False,
 
  # Parametros de figura final
  ncolumnas = 3,
  nfilas    = 1,
  variables = var_pd,
  dim_grafico_total = (30, 6),
  ncol_leyenda = 5,
  vspacio=-0.25,
  hspacio=4,
  tamaño_etiqueta = 14 
)

In [0]:
pd_rbm='PD_RBM'
pd_dep='PD_APP_HIP_TRDPYME_2Q24_REP_FAB' 
pd_vig='PD_APP_HIP_TRDPYME_2Q24_REP_SALESFORCE'
pd_trc='XB_ADC_POND85_F'
tipo_banda = 'Jeffrey'#Jeffrey Vasiseck
codmes_default = 202307
columna_monto='MTOAPROBADO_SOLES'
titulo='Calibración Applicant App Tradicional'
condiciones= "CODMES >= 201901 AND DEF_120_BLOQ_REF_0 = 0"

In [0]:
# Objeto de Seguimiento
Obj_Seg1 = rmm.MonitorScore(
  nombre_modelo         = 'Personas Modelo Applicant Tradicional',
  codigo_modelo         = 'MOD-BCP-20658',
  tier                  = 'II',
  detalle_seguimiento   = 'Seguimiento Applicant Tradicional',
  mes_seguimiento       = '202504',
  base_monitoreo        = Obj_Seg.base_monitoreo,
  pd1                   = pd_dep,
  pd2                   = pd_rbm,
  # pd3                   = pd_dep,
  monto_credito         = columna_monto,
  query_univ            = "DEF_120_BLOQ_REF_0=0 and codmes >=202201 AND PD_APP_HIP_TRDPYME_2Q24_REP_FAB IS NOT NULL and ACT_HIP IN ('EMISION MEDIO DE PAGO', 'DESEMBOLSO', 'ANTES')",
  bandas                = tipo_banda,
  codmes_default        = codmes_default,
  meses_ventana_target  = 24,
  meses_atraso_target   = 4
)

In [0]:
Obj_Seg_cal, rc_1, rm_1, fc_1, fm_1 = Obj_Seg1.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES>=202106",
  pd_seg             = pd_dep,
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 0,
  rd_aux             = 'RD21',
  rd_aux2            = 'RD12', 
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = "Best_proy",
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'det1_xc',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "np.where(calxmontos['CODMES'].isin([202307, 202306, 202311]), met2_xm + 0.003, met2_xm)",
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det1_xm',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = titulo,
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.05,
  eje_y_xm           = 0.05,
  dim_grafico        = (35, 8),
  punt_mora          = 85, #Tamaño de puntos de mora temprana
  etiquetas          =True,
  pos_etiquetas_xm   = 10,
  pos_etiquetas_xc   = 10,
  tamaño_etiqueta    = 14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
rc_1

In [0]:
Obj_Seg.base_monitoreo.createOrReplaceTempView('hip_app_hippyme')
spark.sql('''
    CREATE OR REPLACE TABLE
        catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_hm_app_hippyme_f
    USING DELTA LOCATION 
        'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/T39290/Seguimiento/02_Applicant_Hipotecario/hm_app_hippyme_F'
    AS SELECT * FROM hip_app_hippyme
    ''')